---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [145]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [146]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [147]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    unitown = pd.read_csv('C:/Users/lcopland/Desktop/Python/Assignment4/university_towns.txt', sep='\n', header=None, names=['RegionName'])
    unitown['State'] = np.where(unitown['RegionName'].str.contains('edit'), unitown['RegionName'], np.NaN)
    #rearrange columns
    unitown = unitown[['State','RegionName']]
    #fill state names with appropriate state
    unitown['State'].fillna(method='ffill',inplace=True)
    #loop through columns and split the uni and state identifiers
    for col in unitown: 
        unitown[col] = unitown[col].str.split('(',expand=True)[0].str.split('[',expand=True)[0].str.rstrip()
    
    #remove where state and region names are the same
    unitown.drop(unitown[unitown['State'] == unitown['RegionName']].index, inplace=True)
    
    return unitown

get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [148]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #columns to use for dataframe
    cols = [4,5,6]    #select columns required for dataframe
    #import dataframe and define dataframe
    gdp = pd.read_excel('C:/Users/lcopland/Desktop/Python/Assignment4/gdplev.xls',sheet_name=0, usecols=cols, header=6, skiprows=213, index_col=0)
    #rename index
    gdp.index.name = 'Fiscal Quater'
    #rename columns
    gdp.columns = ('Current Dollars (GDP)', 'Chained 2009 Dallars (GDP)')
    #create column for differences between periods
    gdp['Diff'] = gdp['Chained 2009 Dallars (GDP)'].diff()
    #identify the decline in GDP
    GDPdecline = gdp.where(gdp['Diff']<0)
    #create a row number to identify consecutive periods for the start of recession 
    GDPdecline['RN'] = np.arange(len(gdp))
    #drop NaNs
    GDPdecline = GDPdecline.dropna()
    #create a difference between remaining row numbers (compare row with next row)
    GDPdecline['RNDiff'] = GDPdecline['RN'].diff(periods=-1)
    #identify where recession starts
    RecStart = GDPdecline.where(GDPdecline['RNDiff'] == -1)
    #return index of recession start GDP
    return RecStart['RNDiff'].idxmin()
get_recession_start()

'2008q3'

In [149]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    #columns to use for dataframe
    cols = [4,5,6]    #select columns required for dataframe
    gdp = pd.read_excel('C:/Users/lcopland/Desktop/Python/Assignment4/gdplev.xls',sheet_name=0, usecols=cols, header=6, skiprows=213, index_col=0)
    gdp.index.name = 'Fiscal Quater'
    gdp.columns = ('Current Dollars (GDP)', 'Chained 2009 Dallars (GDP)')
    gdp['Diff'] = gdp['Chained 2009 Dallars (GDP)'].diff()
    #identify GDP incline as greater than 0 and from a period after recession had started
    GDPincline = gdp.where((gdp['Diff']>0) & (gdp.index > get_recession_start())) 
    GDPincline['RN'] = np.arange(len(gdp))
    GDPincline = GDPincline.dropna()

    GDPincline['RNDiff'] = GDPincline['RN'].diff(periods=1)

    RecEnd = GDPincline.where(GDPincline['RNDiff'] == 1)

    return RecEnd['RNDiff'].idxmax()
       
get_recession_end()

'2009q4'

In [260]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    #columns to use for dataframe
    cols = [4,5,6]    #select columns required for dataframe
    gdp = pd.read_excel('C:/Users/lcopland/Desktop/Python/Assignment4/gdplev.xls',sheet_name=0, usecols=cols, header=6, skiprows=213, index_col=0)
    gdp.index.name = 'Fiscal Quater'
    gdp.columns = ('Current Dollars (GDP)', 'Chained 2009 Dollars (GDP)')

    #bottom = gdp.between(get_recession_start(),get_recession_end())
    #bottom = gdp.where((gdp.index >= get_recession_start()) & (gdp.index <= get_recession_end()))

    start_index = gdp.loc[gdp.index == get_recession_start()].index
    end_index = gdp.loc[gdp.index == get_recession_end()].index
    gdp['Quater'] = gdp.index
    bottom = gdp.where((gdp['Quater'] > get_recession_start()) & (gdp['Quater'] < get_recession_end()))
    bottom = bottom.dropna() 
    
    return bottom['Chained 2009 Dollars (GDP)'].idxmin()
get_recession_bottom()

'2009q2'

In [302]:
def convert_columns_to_quarters(period: str):

    charsToRemove = '[]'

    date = str(period.split('-'))
    date = date.strip('[%s]' % charsToRemove)
    month = int(date[9:11])
    quarter = int((month - 1) / 3) + 1

    return date[1:5] + 'q' + str(quarter)


In [306]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv('C:/Users/lcopland/Desktop/Python/Assignment4/City_Zhvi_AllHomes.csv')

    #Drop Columns that are not needed
    housing = housing.drop(housing.columns[[0,3,4,5]], axis=1)
    start = housing.columns.get_loc('1996-04')
    end = housing.columns.get_loc('2000-01')
    housing = housing.drop(housing.columns[start:end],axis=1)

    #sort multi-index
    columnNames = housing.columns[2:]
    #housing = housing.set_index(['State','RegionName'],inplace=True)

    #convert coulmn to period from dates
    StringNames = ['State','RegionName']
    

#    for i in columnNames:
#        #StringNames.append(str(i.split('-')))
#        date = str(i.split('-'))
#        date = date.strip('[%s]' % charsToRemove)
#        month = int(date[9:11])
#        quarter = int((month - 1) / 3) + 1
#        quarter = date[1:5] + 'q' + str(quarter)
#        StringNames.append(quarter)
#        StringNames = list(dict.fromkeys(StringNames))
    
    #housing = housing.groupby(convert_columns_to_quarters, axis=1).mean()

    
    return housing

convert_housing_data_to_quarters()

,RegionName,State,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,Los Angeles,CA,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,Chicago,IL,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,Philadelphia,PA,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,Phoenix,AZ,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
5,Las Vegas,NV,131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
6,San Diego,CA,219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
7,Dallas,TX,85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
8,San Jose,CA,364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700
9,Jacksonville,FL,88000.0,88800.0,89000.0,88900.0,89600.0,90600.0,91200.0,91700.0,...,132000,132500,133100,133900,134900,136000,137200,138400,139500,140300


In [152]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"